## 01 - Fit GAM Model for Age

## Imports

In [60]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [61]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from pygam import LinearGAM, s, f
from sklearn.model_selection import LeaveOneGroupOut

## Prepare Dataset

In [62]:
path_to_feats = 'C:\\Grad_School\\Code_and_software\\Py_code\\All_features_2023\\All_features_2023\\Juvenile_bird_analysis\\'

### Load Duke Data

In [63]:
all_feats = pd.read_csv(path_to_feats + "Duke_all_features_morning.csv").drop(columns= 'Unnamed: 0')

### Load UTSW Data

In [64]:
all_feats_utsw = pd.read_csv( path_to_feats + "utsw_all_features_morning.csv").drop(columns= 'Unnamed: 0')

### Load extra FP1 birds

In [65]:
acoustics = pd.read_csv('C:\\Grad_School\\Code_and_software\\Py_code\\All_features_2023\\All_features_2023\\Juvenile_bird_analysis\\FP1_normal_morning_only\\morning_Juvenile_acoustic_features.csv').drop(columns = 'Unnamed: 0')
syntax = pd.read_csv("C:\\Grad_School\\Code_and_software\\Py_code\\All_features_2023\\All_features_2023\\Juvenile_bird_analysis\\FP1_normal_morning_only\\morning_syntax_features_whisper_UMAP.csv").drop(columns = 'Unnamed: 0')
rhythm_spectrum_feats = pd.read_csv("C:\\Grad_School\\Code_and_software\\Py_code\\All_features_2023\\All_features_2023\\Juvenile_bird_analysis\\FP1_normal_morning_only\\morning_rhythm_spectrum_features.csv").drop(columns = 'Unnamed: 0')
timing_seg_feats = pd.read_csv("C:\\Grad_School\\Code_and_software\\Py_code\\All_features_2023\\All_features_2023\\Juvenile_bird_analysis\\FP1_normal_morning_only\\morning_syllable_and_gap_duration_entropies.csv").drop(columns = 'Unnamed: 0')



In [66]:
all_feats_new = pd.merge(acoustics, syntax, on =["Bird_ID", 'age'])
all_feats_new = pd.merge(all_feats_new, rhythm_spectrum_feats, on = ["Bird_ID", 'age'])
all_feats_new = pd.merge(all_feats_new, timing_seg_feats, on = ["Bird_ID", 'age'])
all_feats_new['colony'] = 'fp1'

In [67]:
all_feats = pd.concat([all_feats, all_feats_utsw, all_feats_new])

In [68]:
all_feats = all_feats[~all_feats.Bird_ID.isin(['S833', 'Y856', 'O833', 'notag833'])]

In [69]:
all_feats.head(3)

,Goodness_mean_median,Goodness_mean_min,Goodness_mean_max,Mean_frequency_mean_median,Mean_frequency_mean_min,Mean_frequency_mean_max,Entropy_mean_median,Entropy_mean_min,Entropy_mean_max,Amplitude_mean_median,...,entropy_rate,entropy_rate_norm,num_syllables,mean_repetition_length,CV_repetition_length,entropy,CV_peak_rhythm_spectrum_frequency,syllable_duration_entropy,gap_duration_entropy,colony
0,0.172179,0.158379,0.212550,3012.855881,1946.598471,3395.702464,-2.095430,-2.361755,-1.726711,79.752925,...,2.114136,0.753070,6,1.419854,0.531558,-0.153195,12.941768,0.750091,0.753782,duke
1,0.167118,0.146229,0.319930,3054.320470,2502.096118,3917.514683,-2.401353,-2.945930,-1.702948,81.314262,...,1.474414,0.443843,9,1.136691,0.302211,-0.157250,11.456134,0.695547,0.687156,duke
2,0.170937,0.155001,0.375353,3256.761069,2438.043051,3734.626728,-2.150781,-2.790517,-1.603982,80.121236,...,1.354237,0.377755,11,1.746479,0.396831,-0.158022,13.630966,0.688570,0.697006,duke


### Drop Amplitude Features

In [70]:
amplitude_features = all_feats.columns[['Amplitude' in x for x in all_feats.columns.tolist()]]

all_feats = all_feats.drop(columns= amplitude_features)

### Normalize Feature Set

In [71]:
full_feats_only = np.array(all_feats.drop(columns = ['Bird_ID', 'age', 'entropy_rate', 'colony', 'num_syllables']))

In [72]:
full_feats_only.shape

(43, 43)

In [73]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
full_feats_only_norm = scaler.fit_transform(full_feats_only)
y = all_feats.age


In [74]:
y = np.array(y).flatten()

## Selecting Features with Mutual Information

In [75]:
mutual_inf_score = mutual_info_regression(full_feats_only_norm, y)
mi_df = pd.DataFrame({'score': mutual_inf_score, 
              'feature' : all_feats.drop(columns = ['Bird_ID', 'age', 'entropy_rate', 'colony', 'num_syllables']).columns})
mi_df.sort_values('score', ascending=False)

,score,feature
33,0.696542,duration_CV_median
26,0.614568,Entropy_CV_max
34,0.580918,duration_CV_min
41,0.497359,syllable_duration_entropy
15,0.447495,duration_mean_median
36,0.410846,entropy_rate_norm
22,0.401133,Mean_frequency_CV_min
42,0.351819,gap_duration_entropy
18,0.298546,Goodness_CV_median
24,0.298250,Entropy_CV_median


In [76]:
mi_df = mi_df[mi_df.score > 0.05]
print(len(mi_df))

feature_indices = mi_df.index.values

24


If we set a cutoff of 0.05 for mutual information, we narrow down this feature set from 44 features to 24 features. Now, among these 24 features, which combination gives us the best age prediction results?

## Forward Feature Selection

In [77]:
groups = all_feats.Bird_ID
groups = groups.reset_index().drop(columns = 'index')
groups.values.flatten()

array(['grn394', 'grn394', 'grn394', 'grn395', 'grn395', 'grn395',
       'grn475', 'grn475', 'grn397', 'grn397', 'sil469', 'sil469',
       'grn394', 'grn395', 'grn397', 'grn475', 'sil469', 'grn394',
       'grn475', 'grn397', 'sil469', 'grn395', 'S855', 'S855', 'S855',
       'S855', 'S855', 'Y855', 'Y855', 'Y855', 'Y855', 'Y855', 'Y855',
       'S856', 'S856', 'S856', 'S856', 'S856', 'S856', 'O434', 'O440',
       'R425', 'R469'], dtype=object)

In [80]:
n_features = len(feature_indices)
selected_features = []
remaining_features = feature_indices
best_scores = []

#create bird-fold cross validation object
logo = LeaveOneGroupOut()

while len(remaining_features) > 0:
    scores_per_feature = []

    for feature in remaining_features:
        current_features = selected_features + [feature]

        #create terms for model
        terms = [s(i) for i in current_features]
        all_terms = terms[0]
        for term in terms[1:]:
            all_terms = all_terms + term

        #loop over splits
        splits = logo.split(full_feats_only_norm, groups = all_feats.Bird_ID)

        #initialize empty arrays to store predictions
        y_preds = np.array([])
        y_tests = np.array([])
        Bird_IDs = np.array([])
        #train and predict for each split in cross validation object
        for train_index, test_index in splits:
            #print(groups.iloc[test_index].values[0])

            X_train, X_test = full_feats_only_norm[(train_index)], full_feats_only_norm[(test_index)]
            y_train, y_test = y[(train_index)], y[(test_index)]

            gam = LinearGAM(all_terms)

            gam.fit(X_train, y_train)
            y_pred = gam.predict(X_test)
            
            y_preds = np.concatenate([y_preds, y_pred])
            y_tests = np.concatenate([y_tests, y_test])
            Bird_IDs = np.concatenate([Bird_IDs, groups.iloc[test_index].values.flatten()])
            
        #calculate MSE 
        MSE = np.array([mean_squared_error(y_tests, y_preds)])
        scores_per_feature.append(np.mean(MSE))

    #find best feature of this round
    best_feature_idx = np.argmin(scores_per_feature)
    best_feature = remaining_features[best_feature_idx]
    best_scores.append(scores_per_feature[best_feature_idx])

    #update list of selected and remaining features
    selected_features.append(best_feature)
    remaining_features = np.delete(remaining_features, np.where(remaining_features == best_feature))

    print(f"Selected feature {best_feature} with score {scores_per_feature[best_feature_idx]}")

    


Selected feature 41 with score 101.36566959534169
Selected feature 36 with score 92.96485840509465
Selected feature 19 with score 86.9160621652485
Selected feature 33 with score 93.74331606358272
Selected feature 16 with score 87.85738058038832
Selected feature 24 with score 76.29152939271998
Selected feature 34 with score 74.76483831036948
Selected feature 22 with score 75.28366006867242
Selected feature 18 with score 76.83576831716299
Selected feature 15 with score 80.28264448182053
Selected feature 29 with score 82.37114424371258
Selected feature 7 with score 86.14922254742274
Selected feature 26 with score 91.77539970773546
Selected feature 35 with score 97.6855810576588
Selected feature 10 with score 103.0513643048036
Selected feature 39 with score 110.04506241270661
Selected feature 17 with score 116.93944641607163
Selected feature 1 with score 123.982265948319
Selected feature 21 with score 132.0435747873107
Selected feature 23 with score 140.5104078839885
Selected feature 9 wit

In [30]:
best_n_feats = np.argmin(best_scores) + 1
best_n_feats

7

In [31]:
features_to_include = selected_features[:best_n_feats]

In [32]:
features_to_include

[41, 36, 19, 33, 16, 24, 34]

In [33]:
mi_df.loc[mi_df.index.isin(features_to_include)]

,score,feature
16,0.137339,duration_mean_min
19,0.331587,Goodness_CV_min
24,0.326210,Entropy_CV_median
33,0.697307,duration_CV_median
34,0.593321,duration_CV_min
36,0.401300,entropy_rate_norm
41,0.550811,syllable_duration_entropy
